
# Perform full SCA analysis

This example shows the full process to perform a complete coevolution
analysis in order to detect protein sectors from data importation, MSA
filtering, computation of positional and joint amino acid frequencies,
and computation of the SCA coevolution matrix.

The matrix is then decomposed into principal components and independent
component analysis is performed.

In the end, we export a fasta file of the residues contributing to the first
independent component.


In [ ]:
# Author: Margaux Jullien <margaux.jullien@univ-grenoble-alpes.fr>
#         Nelle Varoquaux <nelle.varoquaux@univ-grenoble-alpes.fr>
#         Ivan Junier <ivan.junier@univ-grenoble-alpes.fr>
# License: TBD

Import necessary



In [ ]:
from cocoatree.io import load_MSA, export_fasta
from cocoatree.msa import filter_gap_seq, filter_gap_pos, seq_weights
from cocoatree.statistics.position import aa_freq_at_pos, background_freq
from cocoatree.statistics.pairwise import aa_joint_freq, compute_sca_matrix, \
    compute_seq_identity
from cocoatree.deconvolution import eigen_decomp, compute_ica, chooseKpos, \
    icList
from cocoatree.randomize import randomization
import matplotlib.pyplot as plt
import numpy as np

## Import MSA



In [ ]:
seq_id, sequences = load_MSA("data/s1Ahalabi_1470.an", format="fasta")
n_pos, n_seq = len(sequences[0]), len(sequences)

print(f"The loaded MSA has {n_seq} sequences and {n_pos} positions.")

## MSA filtering

### Filter overly gapped positions



In [ ]:
filt_seqs, pos_kept = filter_gap_pos(sequences, threshold=0.4)
n_pos_kept = len(pos_kept)
print(f"After filtering, we have {n_pos_kept} effective sequences.")

### Filter overly gapped sequences



In [ ]:
seq_id_kept, seq_kept = filter_gap_seq(seq_id, filt_seqs, threshold=0.2,
                                       filtrefseq=False)

## Compute the matrix of pairwise sequence identity



In [ ]:
sim_matrix = compute_seq_identity(seq_kept)

fig, ax = plt.subplots()
m = ax.imshow(sim_matrix, vmin=0, vmax=1, cmap='inferno')
ax.set_xlabel("sequences", fontsize=10)
ax.set_ylabel("sequences", fontsize=10)
ax.set_title('Matrix of pairwise sequence identity', fontweight="bold")
cb = fig.colorbar(m)
cb.set_label("Pairwise sequence identity", fontweight="bold")
plt.show()

Compute sequence weights



In [ ]:
weights, n_eff_seq = seq_weights(sim_matrix)
print(f"Number of effective sequences {n_eff_seq}")

compute allele frequencies



In [ ]:
aa_freq = aa_freq_at_pos(seq_kept, lambda_coef=0.03, weights=weights)

Compute background frequencies



In [ ]:
background_frequencies = background_freq(aa_freq)

Compute joint allele frequencies



In [ ]:
fijab, fijab_ind = aa_joint_freq(seq_kept, weights=weights, lambda_coef=0.03)

## Compute the SCA coevolution matrix



In [ ]:
Cijab_raw, Cij = compute_sca_matrix(joint_freqs=fijab,
                                    joint_freqs_ind=fijab_ind,
                                    aa_freq=aa_freq,
                                    qa=background_frequencies)

fig, ax = plt.subplots()
im = ax.imshow(Cij, vmin=0, vmax=1.4, cmap='inferno')

ax.set_xlabel('Residue', fontsize=10)
ax.set_ylabel(None)
ax.set_title('Coevolution matrix')
fig.colorbar(im, shrink=0.7)
plt.show()

### Decomposition of the matrix into principal components



In [ ]:
eigenvalues, eigenvectors = eigen_decomp(Cij)

Plot distribution of eigenvalues



In [ ]:
fig, ax = plt.subplots()
ax.hist(eigenvalues, bins=100, color="black")
ax.set_ylabel('Number', fontweight="bold")
ax.set_xlabel('Eigenvalue', fontweight="bold")
plt.show()

### Select number of significant components



In [ ]:
v_rand, l_rand = randomization(seq_kept, Nrep=10,
                               weights=weights, lambda_coef=0.03, kmax=10,
                               metric='SCA',
                               correction=None)
kpos = chooseKpos(eigenvalues, l_rand)
print('kpos = ' + str(kpos))

hist0, bins = np.histogram(l_rand.flatten(), bins=n_pos_kept,
                           range=(0, eigenvalues.max()))
hist1, bins = np.histogram(eigenvalues, bins=n_pos_kept,
                           range=(0, eigenvalues.max()))

fig, ax = plt.subplots()
ax.bar(bins[:-1], hist1, np.diff(bins), color='k')
ax.plot(bins[:-1], hist0/10, 'r', linewidth=3)
ax.set_xlabel('Eigenvalues', fontweight="bold")
ax.set_ylabel('Numbers', fontweight="bold")
plt.show()

### Independent component analysis (ICA)



In [ ]:
independant_components, W = compute_ica(
    eigenvectors, kmax=kpos, learnrate=0.1,
    iterations=100000)

# Plot results
if kpos % 2 != 0:
    print('Uneven number of axes, discard the last one for visual \
          representation')
    kpos -= 2
else:
    kpos -= 1

pairs = [[x, x+1] for x in range(0, kpos, 2)]
ncols = len(pairs)
plt.rcParams['figure.figsize'] = 14, 8
fig, axes = plt.subplots(nrows=2, ncols=len(pairs), tight_layout=True)
for k, [k1, k2] in enumerate(pairs):
    ax = axes[0, k]
    ax.plot(eigenvectors[:, k1], eigenvectors[:, k2], 'ok')
    ax.set_xlabel("eigenvector %i" % (k1+1), fontsize=16)
    ax.set_ylabel("eigenvector %i" % (k2+1), fontsize=16)

    ax = axes[1, k]
    ax.plot(independant_components[:, k1], independant_components[:, k2], 'ok')
    ax.set_xlabel("independant component %i" % (k1+1), fontsize=16)
    ax.set_ylabel("independant component %i" % (k2+1), fontsize=16)
plt.show()

Select residues that significantly contribute to each independent component



In [ ]:
ics, icsize, sortedpos, cutoff, scaled_pdf, all_fits = icList(
    independant_components, kpos, Cij,
    p_cut=0.95)

print('Sizes of the ' + str(kpos) + ' ICs: ' + str(icsize))

Plot coevolution within and between the sectors



In [ ]:
plt.rcParams['figure.figsize'] = 10, 10
plt.subplot(121)
plt.imshow(Cij[np.ix_(sortedpos, sortedpos)], vmin=0, vmax=2,
           interpolation='none', aspect='equal',
           extent=[0, sum(icsize), 0, sum(icsize)], cmap='inferno')
plt.colorbar(shrink=0.25)
line_index = 0
for i in range(kpos):
    plt.plot([line_index + icsize[i], line_index + icsize[i]],
             [0, sum(icsize)], 'w', linewidth=2)
    plt.plot([0, sum(icsize)], [sum(icsize) - line_index,
                                sum(icsize) - line_index], 'w', linewidth=2)
    line_index += icsize[i]
plt.show()

Export fasta files of the sectors for all the sequences



In [ ]:
sector_1_pos = list(pos_kept[ics[0].items])
sector_1 = []
for sequence in range(len(seq_id)):
    seq = ''
    for pos in sector_1_pos:
        seq += sequences[sequence][pos]
    sector_1.append(seq)

export_fasta(sector_1, seq_id, 'sector_1.fasta')
# outpath needs to be defined properly